In [ ]:
# Tell the notebook where to find the pipeline functions
import sys
sys.path.append('/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Lesion_Pipeline/')

In [ ]:
import os
import shutil
from glob import glob 
import GNMPipeline as p

# Add your inputs

In [ ]:
data_dir="/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/RDCRN_test_output/"   #Folder with input files, can use the folder created by Combine script previously
input_type='BIDS' #'BIDS' or 'Folder'


subjects_to_skip=None #['c002', 'c003'] 

mni_software='EasyReg' #'EasyReg', 'ANTs', 'ANTsQuick'

lesion_keyword='segmentation' #i.e. 'segmentation', or None if you do not have any 

IMAGE_TYPES = ['t1', 'T2', 'FLAIR'] #case sensitive, change to match what you used e.g. t1, t1w, TI 
Reg_target_1='t1' #your ideal registration target, case sensitive
Reg_target_2='T2'

# Make dictionary of subjects and sessions

In [ ]:
subject_sessions=p.create_input_dict(data_dir, subjects_to_skip=subjects_to_skip, input_type=input_type)

# Register Co-registration target to MNI, then bring lesion mask and co-registered images into the same space

In [ ]:
#If you want to use EasyReg with the robust setting, consider running the below to premake the segmentations
# This should save time and computational resources
#takes about 1-2 minutes per brain

#If you use this, make sure to uncomment the right lines in the cell below

# source_files=[]
# for subject, sessions in subject_sessions.items():
#     reg_target=None
#     subject_folder = os.path.join(data_dir, subject) 
#     for session in sessions:
#         print(f'*** Processing {subject}: {session} ***')
#         session_folder=os.path.join(subject_folder, session)
        
#         reg_target=p.set_registration_target(glob(f'{session_folder}/COREGISTERED/*.nii*'), Reg_target_1, Reg_target_2)
#         print("Will register", reg_target, "to MNI")
        
            
#         source_file=f'{session_folder}/COREGISTERED/{subject}_{session}_{reg_target}_SkullStripped.nii.gz'
        
#         source_files.append(source_file)
        

# command=p.synthseg_wrapper(source_files)
# job_name = f"SynthSeg"
# job_id=p.submit_slurm_job(job_name, command)
# print("Your SynthSeg job is running:", job_id)

In [ ]:
#Pre-generate SynthSeg for MNI target brain to save time
MNI_target_brain='/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/0_pipeline_scripts/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii.gz'
MNI_synthseg='/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/mni_icbm152_t1_tal_nlin_asym_09c_brain_synthseg.nii.gz'
!mri_synthseg --i {MNI_target_brain} --o {MNI_synthseg} --parc


In [ ]:
mni_reg_dict={}
for subject, sessions in subject_sessions.items():
    reg_target=None
    subject_folder = os.path.join(data_dir, subject) 
    for session in sessions:
        print(f'*** Processing {subject}: {session} ***')
        session_folder=os.path.join(subject_folder, session)
        
        reg_target=p.set_registration_target(glob(f'{session_folder}/COREGISTERED/*.nii*'), Reg_target_1, Reg_target_2)
        print("Will register", reg_target, "to MNI")
        
        if '1' in reg_target:
            target_image='/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/0_pipeline_scripts/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii.gz'
            target_brain_seg= MNI_synthseg #providing this to EasyReg will save you time
        elif '2' in reg_target:
            target_image='/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/0_pipeline_scripts/mni_icbm152_t2_tal_nlin_asym_09c_brain.nii.gz'
            target_brain_seg= MNI_synthseg
        

        if not os.path.exists(f'{session_folder}/MNI_SPACE'):
            os.makedirs(f'{session_folder}/MNI_SPACE')
            
        source_file=f'{session_folder}/COREGISTERED/{subject}_{session}_{reg_target}_SkullStripped.nii.gz'
        # source_synthseg=f'{session_folder}/COREGISTERED/{subject}_{session}_{reg_target}_SkullStripped_synthseg.nii.gz'
        
  
        if lesion_keyword:
            lesion_mask_files=glob(f'{session_folder}/COREGISTERED/*{lesion_keyword}*nii*')
            if lesion_mask_files:
                lesion_mask = lesion_mask_files[0]
            if len(lesion_mask_files) > 1:
                print(f"Warning: Multiple lesion mask files found. Using the first one: {lesion_mask}")
        else:
            lesion_mask = None


       
        other_brains=glob(f'{session_folder}/COREGISTERED/*space-{reg_target}_SkullStripped.nii.gz') 
  
        if other_brains:
            print("Found", len(other_brains), "other brains to warp to MNI space also")
        else:
            print("Found no other brains to warp to MNI space")
        
        print(f'Registering the {reg_target} to MNI using {mni_software}') 
        if mni_software == 'EasyReg':
            command=p.easy_reg(f'{session_folder}/MNI_SPACE', source_file, target_image, target_brain_seg=target_brain_seg, 
                               source_brain_seg=None,lesion_mask=lesion_mask, tag='_EasyReg', other_brains=other_brains, synthseg_robust=False)
            # command=p.easy_reg(f'{session_folder}/MNI_SPACE', source_file, target_image, target_brain_seg=target_brain_seg, 
            #                    source_brain_seg=source_synthseg,lesion_mask=lesion_mask, tag='space-MNI', other_brains=other_brains, synthseg_robust=False)
        elif mni_software == 'ANTs':
            command=p.ants_mni(f'{session_folder}/MNI_SPACE', source_file, target_image, lesion_mask, other_brains, 
                               transform='s', histogram_matching=False, quick=False)
        elif mni_software == 'ANTsQuick':
            command=p.ants_mni(f'{session_folder}/MNI_SPACE', source_file, target_image, lesion_mask, other_brains, 
                               transform='s', histogram_matching=False, quick=True)
        else:
            print("Please set mni_software to 'EasyReg', 'ANTs', or 'ANTsQuick'")
    
            
        job_name = f"MNI-register_{subject}_{session}_{reg_target}"
        job_id=p.submit_slurm_job(job_name, command)
        mni_reg_dict[(subject,session)] = job_id
        

if len(mni_reg_dict) > 0:
    print('You have', len(mni_reg_dict), 'MNI Registration Jobs submitted to SLURM; subject and job IDs are stored in the co_reg_dict')
    print('You can type "squeue -u $USER" into your terminal  or "!squeue -u $USER" in a cell to track SLURM job progress')
    print('You can check the output file matching the jobid in co_reg_dict to see code outputs and any errors')
                                        

# Check your submitted jobs

In [ ]:
print("These are the jobs that should have been submitted:")
print(mni_reg_dict)

In [ ]:
!squeue -u $USER 